<a href="https://colab.research.google.com/github/anishchapagain/OpenLLM/blob/main/chat_with_pdf_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Implementing RAG
Chat with your document

Step 1: Load the Document

In [ ]:
from langchain.document_loaders import PyPDFLoader

Provide doument path and load the object

In [ ]:
pdf_obj = PyPDFLoader("sample_data/Six Python Web frameworks compared.pdf")
pdf_doc = pdf_obj.load()

In [ ]:
pdf_doc2 = PyPDFLoader("sample_data/django-tutorial.pdf").load()

explore the object

In [ ]:
print(dir(pdf_doc))

['__add__', '__class__', '__class_getitem__', '__contains__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__iadd__', '__imul__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__rmul__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', 'append', 'clear', 'copy', 'count', 'extend', 'index', 'insert', 'pop', 'remove', 'reverse', 'sort']


In [ ]:
len(pdf_doc) #  Length of doc (total pages)

6

In [ ]:
# check some meta data from about page

print(f"First page: {pdf_doc[0].metadata} and \nlast page:{pdf_doc[len(pdf_doc)-1].metadata}")

First page: {'source': 'sample_data/Six Python Web frameworks compared.pdf', 'page': 0} and 
last page:{'source': 'sample_data/Six Python Web frameworks compared.pdf', 'page': 5}


In [ ]:
# Lets print some content from the last page

pdf_doc[5].page_content[:200]

'[11] http://www.infoworld.com/d/application-development/pillars-python-django-web-framework-168643[12] http://www.infoworld.com/d/application-development/pillars-python-cubicweb-web-framework-169105[1'

In [ ]:
# Lets print some content from the first page

pdf_doc[0].page_content[:200]

'Published on InfoWorld (http://www.infoworld.com)Home > Test Center > Application Development > Pillars of Python: Six Python Webframeworks... > Pillars of Python: Six Python Web frameworks comparedPi'

Step 2: Break documents into chunks, split documents.

In [ ]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

# Various types of splitter are available for the types of documents

# text_splitter = CharacterTextSplitter(
#     separator="\n",
#     chunk_size=500,
#     chunk_overlap=150,
#     length_function=len
# )

# RecursiveCharacterTextSplitter handles generic text effectively by trying to keep semantically related pieces of text together. It uses a list of characters (like newlines and spaces) to recursively split the text into smaller parts
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 100
)

In [ ]:
texts = text_splitter.split_documents(pdf_doc)

In [ ]:
text_splitter2 = RecursiveCharacterTextSplitter(
    chunk_size = 2000,
    chunk_overlap = 200
)
texts2 = text_splitter.split_documents(pdf_doc2)

In [ ]:
# texts

In [ ]:
len(texts)

17

In [ ]:
texts[0].metadata

{'source': 'sample_data/Six Python Web frameworks compared.pdf', 'page': 0}

Step 3: Embeddings and Vectors
 - Embedding models create a vector representation of a piece of text

In [ ]:
# !pip install -U sentence-transformers
# !pip install -U langchain-huggingface

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

In [ ]:
embeddings_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2') # uses default or provide model name
#embeddings_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
#embeddings = embeddings_model.encode(texts)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
embeddings_model

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-mpnet-base-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [ ]:
embeddings = embeddings_model.embed_documents([t.page_content for t in texts])

In [ ]:
embeddings2 = embeddings_model.embed_documents([t.page_content for t in texts2])

In [ ]:
print(embeddings[1])

[0.04003842547535896, 0.007408869918435812, 0.009532025083899498, -0.028926415368914604, 0.03152168542146683, 0.038307320326566696, 0.00893806666135788, 0.03869820013642311, 0.03240501880645752, -0.035013433545827866, -0.0038236205000430346, -0.019508637487888336, -0.0114468764513731, 0.07430919259786606, 0.01589842699468136, -0.04424750432372093, -0.024317588657140732, 0.009504065848886967, 0.013484138995409012, 0.04304581880569458, 0.0022763381712138653, 0.010166814550757408, -0.04432743042707443, 0.041128940880298615, -0.00927024520933628, 0.000900412502232939, -0.02456771209836006, 0.02430560812354088, -0.035833101719617844, -0.04800298064947128, -0.023996848613023758, -0.02775401622056961, 0.006136573851108551, 0.007041544653475285, 2.528028289816575e-06, -0.019529063254594803, 0.00889536552131176, 0.02000272087752819, -0.015706155449151993, 0.019952429458498955, 0.05538599565625191, 0.015849625691771507, 0.010400401428341866, 0.04181225970387459, 0.015592016279697418, -0.02945589

In [ ]:
print(f"Chunk size: {len(embeddings[0])}")
print(f"Shape: {(len(embeddings), len(embeddings[0]))}")

Chunk size: 768
Shape: (17, 768)


In [ ]:
print(f"Chunk size: {len(embeddings2[0])}")
print(f"Shape: {(len(embeddings2), len(embeddings2[0]))}")

Chunk size: 768
Shape: (22, 768)


Store the vector

In [ ]:
# !pip install -U chromadb

In [ ]:
from langchain.vectorstores import Chroma

persist_directory = 'sample_data/chroma'

vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embeddings_model,
    persist_directory=persist_directory
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [ ]:
vectordb2 = Chroma.from_documents(
    documents=texts2,
    embedding=embeddings_model,
    persist_directory='sample_data/chroma2'
)

In [ ]:
vectordb.persist() # save data or configurations to a persistent storage medium like disk using your providded path or a database

In [ ]:
vectordb2.persist() # save data or configurations to a persistent storage medium like disk using your providded path or a database

In [ ]:
print(vectordb._collection.count()) # Count the collection

17


In [ ]:
print(vectordb2._collection.count()) # Count the collection

22


In [ ]:
print(dir(vectordb))

['_Chroma__query_collection', '_LANGCHAIN_DEFAULT_COLLECTION_NAME', '__abstractmethods__', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_asimilarity_search_with_relevance_scores', '_client', '_client_settings', '_collection', '_cosine_relevance_score_fn', '_embedding_function', '_euclidean_relevance_score_fn', '_get_retriever_tags', '_max_inner_product_relevance_score_fn', '_persist_directory', '_select_relevance_score_fn', '_similarity_search_with_relevance_scores', 'aadd_documents', 'aadd_texts', 'add_documents', 'add_images', 'add_texts', 'adelete', 'afrom_documents', 'afrom_texts', 'aget_by_ids', 'amax_marginal_relevance_search', 'a

### Search methods:
'search' 'similarity_search',
 'similarity_search_by_vector',
 'similarity_search_with_relevance_scores',
 'similarity_search_with_score',
 'update_document'

In [ ]:
docs = vectordb.similarity_search("List all the links available", k=3) # k: number of results

In [ ]:
docs

[Document(metadata={'page': 3, 'source': 'sample_data/Six Python Web frameworks compared.pdf'}, page_content='Web.py 0.35[13]Any 2.x Pythonback to 2.3but includesthe CherryPyWeb servergoverned bythe CherryPylicenseCookbook, APIreference, andcategorized codeexamplesAny Web serversupporting CGI,FastCGI, SCGI, orWSGIWeb2py 1.95[9]Python 2.5through 2.7LGPLOnline book,example website,lots of online"quick examples"Any Web serverthat supportsPythonZope 2.13 [7]Depends onZope 2 version,varying fromPython 2.4 forZope 2.11.x toPython 2.7 forZope 2.13.xZope PublicLicense [14]Online bookincludes APIreference,managementinterface guide,templatesreference, andmoreAny Web serverthat supportsPython. Alsoincludes its ownWeb serversuitable forproduction use. Python Web frameworks compared, continued DatabasesupportCachingDebuggingLoggingJavaScriptsupportCubicWeb3.12.5 [12]MySQL,PostgreSQL,SQLite,Microsoft SQLServerPlannedDebug modewill display logmessages ona console,show traceinformation,showgeneratedSQ

In [147]:
# docs2 = vectordb2.similarity_search("python function that takes in a request and returns a response", k=3) # k: number of results
docs2 = vectordb2.similarity_search("provide name of python function that takes in a request and returns a response", k=3) # k: number of results
docs2

[Document(metadata={'page': 21, 'source': 'sample_data/django-tutorial.pdf'}, page_content="Request\tObject\nThe\trequest\tobject\tcontains\ta\tlot\tof\tinformation\tthat\twe\tcan\taccess\tin\tthe\tview.\nFor\texample\trequest.META\tgives\tall\tthe\theader\tinformation.\tIt\tis\ta\tpython\tdictionary.\ndef\thome(request):\n\t\t\t\ttest\t=\trequest.META\n\t\t\t\treturn\trender(request,\t'home.html',\t{'test':\ttest})\n<div>\n\t\t{%\tfor\tkey,value\tin\ttest.items\t%}\n\t\t\t\t<p>{{key}}\t:\t{{value}}</p>\n\t\t{%\tendfor\t%}\n</div>\ntest.items()\tgives\tus\ta\tlist\tof\ttuples\tin\tthe\tdictionary\ttest.\tIn\ta\ttemplate,\twe\tdon't\tneed\tto\tuse\nthe\t()\tto\tcall\ta\tfunction\nRequest\tObject\n22"),
 Document(metadata={'page': 6, 'source': 'sample_data/django-tutorial.pdf'}, page_content='Views\nA\tview\tis\ta\tpython\tfunction\tthat\ttakes\tin\ta\trequest\tand\treturns\ta\tresponse.\nThe\trequest\tobject\tcontains\tmetadata\tabout\tthe\trequest.\tThe\tfirst\targument\tto\tthe\tview\

In [148]:
for doc2 in docs2:
    print(doc2.page_content[:100])

Request	Object
The	request	object	contains	a	lot	of	information	that	we	can	access	in	the	view.
For	
Views
A	view	is	a	python	function	that	takes	in	a	request	and	returns	a	response.
The	request	object
Django	Forms
Create	a	file	called	
forms.py
	in	myapp	folder
We	can	create	a	form	from	scratch,	but	


In [ ]:
len(docs)

3

In [ ]:
for doc in docs:
    print(doc.page_content[:100])

Web.py 0.35[13]Any 2.x Pythonback to 2.3but includesthe CherryPyWeb servergoverned bythe CherryPylic
[11] http://www.infoworld.com/d/application-development/pillars-python-django-web-framework-168643[1
Django 1.3 [11]MySQL,PostgreSQL,SQLite,Oracle; third-party supportfor otherdatabasesYes, bothat them


In [ ]:
search_oracle = vectordb.similarity_search("how often the term 'oracle' is mentioned", k=3) # k: number of results

In [ ]:
for oracle in search_oracle:
    print(oracle.page_content[:100])

Django 1.3 [11]MySQL,PostgreSQL,SQLite,Oracle; third-party supportfor otherdatabasesYes, bothat them
Published on InfoWorld (http://www.infoworld.com)Home > Test Center > Application Development > Pill
DatabaseAbstraction Layer (DAL). Web2py's rendering system will try to find a view that --depending 


In [ ]:
# prompt: asks what is the is a descendant of Zope

descendants = vectordb.similarity_search("what is a descendant of Zope", k=3) # k: number of results
for descendant in descendants:
    print(descendant.page_content)


Python Web frameworks: Light, heavy, and in betweenZope 2 [7] is a descendant of Zope, the great-granddaddy of all Python Web applicationtools and an important branch in the Python family tree. Zope 2's ancestry, then, goes backto the mid-1990s, and Zope's continued popularity is testament to its solidity.Consequently, one huge advantage to choosing Zope 2 is that you can call on 15 years ofcode development and documentation.Zope 2's "object publishing" system fits well into an object-oriented development mind-setand mitigates somewhat the learning curve you'd have to climb if you picked a frameworkthat exposes your application to more of the more painful aspects of Web development.Zope 2 does present a formidable learning curve of its own -- but you'll at least enjoy thestability that a 15-year pedigree confers.The Web2py [9] framework is an abstraction paradise. Databases hide behind a DatabaseAbstraction Layer (DAL). Web2py's rendering system will try to find a view that --depending

In [ ]:
# prompt: list all the URLs that contains word 'pillars-'

urls = vectordb.similarity_search("List all the URLs that contains word 'pillars-'", k=3) # k: number of results
for url in urls:
    print(url.page_content)


[11] http://www.infoworld.com/d/application-development/pillars-python-django-web-framework-168643[12] http://www.infoworld.com/d/application-development/pillars-python-cubicweb-web-framework-169105[13] http://www.infoworld.com/d/application-development/pillars-python-webpy-web-framework-169072[14] http://www.zope.org/Resources/License[15] http://www.infoworld.com/d/application-development/pillars-python-six-python-web-frameworks-compared-169442?source=footer[16] http://www.infoworld.com/?source=footer[17] http://www.infoworld.com/d/application-development?source=footer[18] http://www.infoworld.com/t/python?source=footer[19] http://twitter.com/infoworld
theframework. In addition, its laissez-faire attitude can leave you wondering precisely how toproceed to accomplish a particular task -- and, when you manage to accomplish that task,whether your solution was the best. For example, there are two separate means ofdetermining how your application will handle a given URL: the well-known URL

Retrive some contents

In [ ]:
qDjango = "what was originallypublished at InfoWorld"
djangos = vectordb.similarity_search(qDjango, k=3)

In [ ]:
djangos

[Document(metadata={'page': 4, 'source': 'sample_data/Six Python Web frameworks compared.pdf'}, page_content='http://www.infoworld.com/newsletters/subscribe?showlist=infoworld_developer&amp;source=ifwelg_fssr[7] http://www.infoworld.com/d/application-development/pillars-python-zope-2-web-framework-168935[8] http://www.infoworld.com/d/application-development/programming-python-webpy-web-framework-169072[9] http://www.infoworld.com/d/application-development/pillars-python-web2py-web-framework-168920[10] http://www.infoworld.com/d/application-development/pillars-python-pyramid-web-framework-168661'),
 Document(metadata={'page': 0, 'source': 'sample_data/Six Python Web frameworks compared.pdf'}, page_content="Published on InfoWorld (http://www.infoworld.com)Home > Test Center > Application Development > Pillars of Python: Six Python Webframeworks... > Pillars of Python: Six Python Web frameworks comparedPillars of Python: Six Python Webframeworks comparedBy Rick GrehanCreated 2011-08-10 03

In [ ]:
retriever = vectordb.as_retriever()

In [ ]:
docs = retriever.invoke("where to follow InfoWorld.com?")
docs

[Document(metadata={'page': 4, 'source': 'sample_data/Six Python Web frameworks compared.pdf'}, page_content='http://www.infoworld.com/newsletters/subscribe?showlist=infoworld_developer&amp;source=ifwelg_fssr[7] http://www.infoworld.com/d/application-development/pillars-python-zope-2-web-framework-168935[8] http://www.infoworld.com/d/application-development/programming-python-webpy-web-framework-169072[9] http://www.infoworld.com/d/application-development/pillars-python-web2py-web-framework-168920[10] http://www.infoworld.com/d/application-development/pillars-python-pyramid-web-framework-168661'),
 Document(metadata={'page': 5, 'source': 'sample_data/Six Python Web frameworks compared.pdf'}, page_content='[11] http://www.infoworld.com/d/application-development/pillars-python-django-web-framework-168643[12] http://www.infoworld.com/d/application-development/pillars-python-cubicweb-web-framework-169105[13] http://www.infoworld.com/d/application-development/pillars-python-webpy-web-framew

In [155]:
retriever2 = vectordb2.as_retriever()
rdocs2 = retriever2.invoke("A dictionary is nothing but an unordered collection of what pair")
rdocs2

[Document(metadata={'page': 21, 'source': 'sample_data/django-tutorial.pdf'}, page_content="Request\tObject\nThe\trequest\tobject\tcontains\ta\tlot\tof\tinformation\tthat\twe\tcan\taccess\tin\tthe\tview.\nFor\texample\trequest.META\tgives\tall\tthe\theader\tinformation.\tIt\tis\ta\tpython\tdictionary.\ndef\thome(request):\n\t\t\t\ttest\t=\trequest.META\n\t\t\t\treturn\trender(request,\t'home.html',\t{'test':\ttest})\n<div>\n\t\t{%\tfor\tkey,value\tin\ttest.items\t%}\n\t\t\t\t<p>{{key}}\t:\t{{value}}</p>\n\t\t{%\tendfor\t%}\n</div>\ntest.items()\tgives\tus\ta\tlist\tof\ttuples\tin\tthe\tdictionary\ttest.\tIn\ta\ttemplate,\twe\tdon't\tneed\tto\tuse\nthe\t()\tto\tcall\ta\tfunction\nRequest\tObject\n22"),
 Document(metadata={'page': 8, 'source': 'sample_data/django-tutorial.pdf'}, page_content="Template\tLanguage\nWe\thave\tlearnt\tto\tserver\tup\ta\tstatic\tpage\ton\tour\tDjango\tserver.\nLet's\tadd\tsome\tdynamic\tcontent.\tThis\tis\twhere\twe\tinterpolate\tour\tHTML\tfile\twith\tsome\np

In [156]:

for rdoc in rdocs2:
  print(rdoc.page_content)

Request	Object
The	request	object	contains	a	lot	of	information	that	we	can	access	in	the	view.
For	example	request.META	gives	all	the	header	information.	It	is	a	python	dictionary.
def	home(request):
				test	=	request.META
				return	render(request,	'home.html',	{'test':	test})
<div>
		{%	for	key,value	in	test.items	%}
				<p>{{key}}	:	{{value}}</p>
		{%	endfor	%}
</div>
test.items()	gives	us	a	list	of	tuples	in	the	dictionary	test.	In	a	template,	we	don't	need	to	use
the	()	to	call	a	function
Request	Object
22
Template	Language
We	have	learnt	to	server	up	a	static	page	on	our	Django	server.
Let's	add	some	dynamic	content.	This	is	where	we	interpolate	our	HTML	file	with	some
python	variables.	We	pass	the	python	variables	though	a	dictionary.	A	dictionary	is	nothing
but	an	unordered	collection	of	key-value	pairs.
myapp/views.py
from	django.shortcuts	import	render
import	datetime
def	hello(request):
				today	=	datetime.datetime.now().date()
				return	render(request,	'hello.html',	{'t

**Different ways of Retrieval**

In [ ]:
from langchain.retrievers import SVMRetriever
from langchain.retrievers import TFIDFRetriever
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
# Load PDF
loader = PyPDFLoader("sample_data/django-tutorial.pdf")
pages = loader.load()
all_page_text=[p.page_content for p in pages]
joined_page_text=" ".join(all_page_text)

In [ ]:
# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1500,chunk_overlap = 150)
splits = text_splitter.split_text(joined_page_text)

In [ ]:
splits

["1.1\n1.2\n1.3\n1.4\n1.5\n1.6\n1.7\n1.8\n1.9\n1.10\n1.11\n1.12\n1.13\n1.14\n1.15\nTable\tof\tContents\nGetting\tstarted\nStarting\ta\tproject\nCreating\tan\tapp\nThe\tadmin\tinterface\nViews\nTemplate\tLanguage\nModels\nRegex\nQuerySets\nURL\tMapping\nDjango\tForms\nAuthentication\nTemplate\tExtending\nFile\tUpload\nRequest\tObject\n1 Getting\tstarted\twith\tDjango\nWe\tassume\tyou\thave\tthe\tlatest\tversion\tof\tpython\tinstalled.\tAnd\talso\tDjango\tinstalled.\nGetting\tstarted\n2 Starting\ta\tproject\nLet's\tcreate\ta\tproject\tcalled\tmysite.\ndjango-admin\tstartproject\tmysite\nThis\tcommand\twill\tcreate\ta\tfolder\tcalled\tmysite\twith\tall\tthe\tfiles\trequired\tto\tget\tour\tproject\nstarted.\nLet's\tcd\tinto\tour\tdirectory\tto\tstart\tdoing\tsome\twork.\ncd\tmysite\nOpen\tthe\tsettings.py\tfile.\tModify\tthe\tTIME_ZONE\tto\twherever\tyou\tare\tfrom.\nTIME_ZONE\t=\t'Asia/Kolkata'\nAdd\ta\tpath\tfor\tstatic\tfiles\nSTATIC_URL\t=\t'/static/'\nSTATIC_ROOT\t=\tos.path.join(BASE

In [ ]:
# Retrieve
svm_retriever = SVMRetriever.from_texts(splits,embeddings_model)
tfidf_retriever = TFIDFRetriever.from_texts(splits)

In [ ]:
# Create a question for the loader
# question = "What are the key features of Django?"
# question = "Where can we access the can access the admin interface?"
question = "What is a python function that takes in a request and returns a response?"

In [ ]:
docs_svm=svm_retriever.invoke(question)
docs_svm[0]

Document(page_content='enctype\t=\t"multipart/form-data"\nTo\tdisplay\tthe\timage\tin\tthe\ttemplate\n{%\tif\tpost.picture\t%}\n<img\tsrc="{{\tpost.picture.url\t}}">\nTo\treceive\tthe\tfiles\tin\tthe\tview,\twe\tneed\tto\tuse\tREQUEST.FILES\nform\t=\tPostForm(request.POST,\trequest.FILES)\nrequest.FILES\n\tis\ta\tdictionary\tlike\tobject\tcontaing\tall\tthe\tuploaded\tfiles.\nNote:\tTo\tdeal\twith\timages,\tpython\trequires\ta\tlibrary\tcalled\tPillow.\nsudo\tpip\tinstall\tpillow\nFile\tUpload\n20 File\tUpload\n21 Request\tObject\nThe\trequest\tobject\tcontains\ta\tlot\tof\tinformation\tthat\twe\tcan\taccess\tin\tthe\tview.\nFor\texample\trequest.META\tgives\tall\tthe\theader\tinformation.\tIt\tis\ta\tpython\tdictionary.\ndef\thome(request):\n\t\t\t\ttest\t=\trequest.META\n\t\t\t\treturn\trender(request,\t\'home.html\',\t{\'test\':\ttest})\n<div>\n\t\t{%\tfor\tkey,value\tin\ttest.items\t%}\n\t\t\t\t<p>{{key}}\t:\t{{value}}</p>\n\t\t{%\tendfor\t%}\n</div>\ntest.items()\tgives\tus\ta\tli

In [ ]:
docs_svm[0].page_content

'enctype\t=\t"multipart/form-data"\nTo\tdisplay\tthe\timage\tin\tthe\ttemplate\n{%\tif\tpost.picture\t%}\n<img\tsrc="{{\tpost.picture.url\t}}">\nTo\treceive\tthe\tfiles\tin\tthe\tview,\twe\tneed\tto\tuse\tREQUEST.FILES\nform\t=\tPostForm(request.POST,\trequest.FILES)\nrequest.FILES\n\tis\ta\tdictionary\tlike\tobject\tcontaing\tall\tthe\tuploaded\tfiles.\nNote:\tTo\tdeal\twith\timages,\tpython\trequires\ta\tlibrary\tcalled\tPillow.\nsudo\tpip\tinstall\tpillow\nFile\tUpload\n20 File\tUpload\n21 Request\tObject\nThe\trequest\tobject\tcontains\ta\tlot\tof\tinformation\tthat\twe\tcan\taccess\tin\tthe\tview.\nFor\texample\trequest.META\tgives\tall\tthe\theader\tinformation.\tIt\tis\ta\tpython\tdictionary.\ndef\thome(request):\n\t\t\t\ttest\t=\trequest.META\n\t\t\t\treturn\trender(request,\t\'home.html\',\t{\'test\':\ttest})\n<div>\n\t\t{%\tfor\tkey,value\tin\ttest.items\t%}\n\t\t\t\t<p>{{key}}\t:\t{{value}}</p>\n\t\t{%\tendfor\t%}\n</div>\ntest.items()\tgives\tus\ta\tlist\tof\ttuples\tin\tth

In [ ]:
docs_tfidf=tfidf_retriever.invoke(question)
docs_tfidf[0]

Document(page_content="To\tcreate\ta\tnew\tapp\npython\tmanage.py\tstartapp\tmyapp\nThis\tcreates\ta\tnew\tfolder\twithin\tour\tproject\tfolder\twith\tall\tthe\tfiles\trequired\tfor\tthe\tapp.\nWe\tneed\tto\tadd\tthe\tname\tof\tthe\tapp\tin\tour\tproject\tsettings.py\tfile.\nINSTALLED_APPS\t=\t[\n\t\t\t\t'django.contrib.admin',\n\t\t\t\t'django.contrib.auth',\n\t\t\t\t...,\n\t\t\t\t'myapp'\n]\nDjango\tUses\tthe\t\nModel-View-Template\n\tframework\nModel\n\tis\ta\tway\tto\tstore\tour\tdata.\nView\n\textracts\tdata\tfrom\tthe\tmodel,\tand\tpasses\tit\ton\tto\tthe\ttemplate\nTemplate\n\tdisplays\tthe\tdata.\nCreating\tan\tapp\n5 The\tadmin\tinterface\nWe\tcan\taccess\tthe\tadmin\tinterface\tat\t\nhttp://localhost:8000/admin\nIt\tallows\tto\taccess\tthe\tmodels\t(Data)\tin\tour\tproject\nBut\tfirst,\twe\tneed\tto\tcreate\ta\tsuperuser\tto\tbe\table\tto\taccess\tour\tadmin\tinterface\npython\tmanage.py\tcreatesuperuser\nThe\tadmin\tinterface\twill\tenable\tyou\tto\tperform\tall\tthe\tbasic\

In [ ]:
docs_tfidf[0].page_content

"To\tcreate\ta\tnew\tapp\npython\tmanage.py\tstartapp\tmyapp\nThis\tcreates\ta\tnew\tfolder\twithin\tour\tproject\tfolder\twith\tall\tthe\tfiles\trequired\tfor\tthe\tapp.\nWe\tneed\tto\tadd\tthe\tname\tof\tthe\tapp\tin\tour\tproject\tsettings.py\tfile.\nINSTALLED_APPS\t=\t[\n\t\t\t\t'django.contrib.admin',\n\t\t\t\t'django.contrib.auth',\n\t\t\t\t...,\n\t\t\t\t'myapp'\n]\nDjango\tUses\tthe\t\nModel-View-Template\n\tframework\nModel\n\tis\ta\tway\tto\tstore\tour\tdata.\nView\n\textracts\tdata\tfrom\tthe\tmodel,\tand\tpasses\tit\ton\tto\tthe\ttemplate\nTemplate\n\tdisplays\tthe\tdata.\nCreating\tan\tapp\n5 The\tadmin\tinterface\nWe\tcan\taccess\tthe\tadmin\tinterface\tat\t\nhttp://localhost:8000/admin\nIt\tallows\tto\taccess\tthe\tmodels\t(Data)\tin\tour\tproject\nBut\tfirst,\twe\tneed\tto\tcreate\ta\tsuperuser\tto\tbe\table\tto\taccess\tour\tadmin\tinterface\npython\tmanage.py\tcreatesuperuser\nThe\tadmin\tinterface\twill\tenable\tyou\tto\tperform\tall\tthe\tbasic\tCRUD\t(Create,\tRead,